In [ ]:
# =======================
# Coffee Sales Full Project – Single Script
# =======================

# If missing libraries, uncomment the next line in Colab or Jupyter:
# !pip install pandas numpy matplotlib seaborn scikit-learn joblib

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error
import joblib
from pathlib import Path

# ---- Load and clean data ----
DATA_PATH = Path('index.csv')  # ensure your index.csv is in the same folder

df = pd.read_csv(DATA_PATH)

# normalize column names
df.columns = [c.strip().lower().replace(' ', '_') for c in df.columns]

# date/datetime conversion
if 'date' in df.columns:
    df['date'] = pd.to_datetime(df['date'], errors='coerce')
if 'datetime' in df.columns:
    df['datetime'] = pd.to_datetime(df['datetime'], errors='coerce')
else:
    df['datetime'] = df['date']

# numeric money
if 'money' in df.columns:
    df['money'] = pd.to_numeric(df['money'], errors='coerce')

# feature engineering
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.strftime('%Y-%m')
df['day_of_week'] = df['date'].dt.day_name()
df['hour'] = df['datetime'].dt.hour

if 'coffee_name' in df.columns:
    df['coffee_name'] = df['coffee_name'].astype(str).str.strip()

df = df.dropna(subset=['date','money']).reset_index(drop=True)

print("Rows:", len(df))
print("Missing values:\n", df.isnull().sum())
print("Top coffee types:\n", df['coffee_name'].value_counts().head(10))

# ---- EDA plots ----
sns.set_style("whitegrid")

# Revenue by product
revenue = df.groupby('coffee_name')['money'].sum().sort_values(ascending=False)
plt.figure(figsize=(8,6))
sns.barplot(x=revenue.values, y=revenue.index)
plt.title('Revenue by Coffee Type')
plt.xlabel('Revenue')
plt.show()

# Monthly transactions
monthly = df.groupby('month').size().reset_index(name='transactions')
plt.figure(figsize=(10,4))
sns.lineplot(data=monthly, x='month', y='transactions', marker='o')
plt.title('Monthly Transactions')
plt.xticks(rotation=45)
plt.show()

# Hourly profile
hourly = df.groupby('hour').size().reindex(range(0,24), fill_value=0)
plt.figure(figsize=(10,4))
sns.barplot(x=hourly.index, y=hourly.values)
plt.title('Transactions by Hour')
plt.xlabel('Hour of Day')
plt.show()

# ---- Simple Model for Daily Sales ----
daily = df.groupby('date').agg({'money':'sum'}).rename(columns={'money':'sales'})
daily = daily.sort_index().reset_index()

# create lag features
for lag in range(1,8):
    daily[f'lag_{lag}'] = daily['sales'].shift(lag)
daily['dayofweek'] = daily['date'].dt.weekday
daily = daily.dropna().reset_index(drop=True)

X = daily.drop(columns=['date','sales'])
y = daily['sales']

tscv = TimeSeriesSplit(n_splits=3)
rmses = []
best_model = None

for train_index, test_index in tscv.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    model = RandomForestRegressor(n_estimators=200, random_state=42)
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    rmse = mean_squared_error(y_test, preds, squared=False)
    rmses.append(rmse)
    best_model = model

print("Cross-val RMSEs:", rmses)

# Save model locally (optional)
joblib.dump(best_model, 'rf_daily_sales.pkl')
print("Model saved to rf_daily_sales.pkl")

print("✅ All done: EDA plots displayed above, model trained & saved.")
